In [1]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from load_dataset import Dataset
import os

import time
# import matplotlib.pyplot as plt
# from PIL import Image
import cv2


In [2]:


class MNIST_PAQ:
	def __init__(self, filename="saved_models", number_of_clients=1, aggregate_epochs=10, local_epochs=5, precision=7, r=1.0):
		self.model = None
		self.criterion = torch.nn.CrossEntropyLoss()
		self.optimizer = None
		self.number_of_clients = number_of_clients
		self.aggregate_epochs = aggregate_epochs
		self.local_epochs = local_epochs
		self.precision = precision
		self.r = r
		self.filename = filename

	def define_model(self):
		self.model = torch.nn.Sequential(
			torch.nn.Conv2d(1, 2, kernel_size=5),
			torch.nn.ReLU(),
			torch.nn.Conv2d(2, 4, kernel_size=7),
			torch.nn.ReLU(),
			torch.nn.Flatten(),
			torch.nn.Linear(1296, 512),
			torch.nn.ReLU(),
			torch.nn.Linear(512, 128),
			torch.nn.ReLU(),
			torch.nn.Linear(128, 32),
			torch.nn.ReLU(),
			torch.nn.Linear(32, 10),
			torch.nn.Softmax(dim=1),
		)		

	def get_weights(self, dtype=np.float32):
		precision = self.precision 
		weights = []
		for layer in self.model:
			try:
				weights.append([np.around(layer.weight.detach().numpy().astype(dtype), decimals=precision), np.around(layer.bias.detach().numpy().astype(dtype), decimals=precision)])
			except:
				continue
		
			
		return np.array(weights)

	def set_weights(self, weights):
		index = 0
		for layer_no, layer in enumerate(self.model):
			try:
				_ = self.model[layer_no].weight
				self.model[layer_no].weight = torch.nn.Parameter(weights[index][0])
				self.model[layer_no].bias = torch.nn.Parameter(weights[index][1])
				index += 1
			except:
				continue

	def average_weights(self, all_weights):
		all_weights = np.array(all_weights)
		all_weights = np.mean(all_weights, axis=0)
		all_weights = [[torch.from_numpy(i[0].astype(np.float32)), torch.from_numpy(i[1].astype(np.float32))] for i in all_weights]
		return all_weights

	def client_generator(self, train_x, train_y):
		number_of_clients = self.number_of_clients
		size = train_y.shape[0]//number_of_clients
		train_x, train_y = train_x.numpy(), train_y.numpy()
		train_x = np.array([train_x[i:i+size] for i in range(0, len(train_x)-len(train_x)%size, size)])
		train_y = np.array([train_y[i:i+size] for i in range(0, len(train_y)-len(train_y)%size, size)])
		train_x = torch.from_numpy(train_x)
		train_y = torch.from_numpy(train_y)
		return train_x, train_y

	def single_client(self, dataset, weights, E):
		self.define_model()
		if weights is not None:
			self.set_weights(weights)
		self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
		for epoch in range(E):
			running_loss = 0
			for batch_x, target in zip(dataset['x'], dataset['y']):
				output = self.model(batch_x)
				loss = self.criterion(output, target)
				self.optimizer.zero_grad()
				loss.backward()
				self.optimizer.step()
				running_loss += loss.item()
			running_loss /= len(dataset['y'])
		weights = self.get_weights()
		return weights, running_loss

	def test_aggregated_model(self, test_x, test_y, epoch):
		acc = 0
		with torch.no_grad():
			for batch_x, batch_y in zip(test_x, test_y):
				y_pred = self.model(batch_x)
				y_pred = torch.argmax(y_pred, dim=1)
				acc += torch.sum(y_pred == batch_y)/y_pred.shape[0]
		torch.save(self.model, "./"+self.filename+"/model_epoch_"+str(epoch+1)+".pt")
		return (acc/test_x.shape[0])
			

	def train_aggregator(self, datasets, datasets_test):
		local_epochs = self.local_epochs
		aggregate_epochs = self.aggregate_epochs
		os.system('mkdir '+self.filename)
		E = local_epochs
		aggregate_weights = None
		for epoch in range(aggregate_epochs):
			all_weights = []
			client = 0
			running_loss = 0
			selections = np.arange(datasets['x'].shape[0])
			np.random.shuffle(selections)
			selections = selections[:int(self.r*datasets['x'].shape[0])]
			clients = tqdm(zip(datasets['x'][selections], datasets['y'][selections]), total=selections.shape[0])
			for dataset_x, dataset_y in clients:
				dataset = {'x':dataset_x, 'y':dataset_y}
				weights, loss = self.single_client(dataset, aggregate_weights, E)
				running_loss += loss
				all_weights.append(weights)
				client += 1
				clients.set_description(str({"Epoch":epoch+1,"Loss": round(running_loss/client, 5)}))
				clients.refresh()
			aggregate_weights = self.average_weights(all_weights)
			self.set_weights(aggregate_weights)
			test_acc = self.test_aggregated_model(datasets_test['x'], datasets_test['y'], epoch)
			print("Test Accuracy:", round(test_acc.item(), 5))
			clients.close()



In [8]:

number_of_clients = 328
aggregate_epochs = 10
local_epochs = 3
r = 0.5
filename = "saved_models"

train_x, train_y, test_x, test_y = Dataset().load_csv()

m = MNIST_PAQ(filename=filename, r=r, number_of_clients=number_of_clients, aggregate_epochs=aggregate_epochs, local_epochs=local_epochs)
train_x, train_y = m.client_generator(train_x, train_y)
m.train_aggregator({'x':train_x, 'y':train_y}, {'x':test_x, 'y':test_y})

  0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_21348\2750871235.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights)
{'Epoch': 1, 'Loss': 2.30127}: 100%|██████████| 164/164 [00:33<00:00,  4.85it/s]


Test Accuracy: 0.10314


{'Epoch': 2, 'Loss': 2.06227}: 100%|██████████| 164/164 [00:40<00:00,  4.05it/s]


Test Accuracy: 0.40439


{'Epoch': 3, 'Loss': 1.81644}: 100%|██████████| 164/164 [00:38<00:00,  4.26it/s]


...

Test Accuracy: 0.83056


{'Epoch': 9, 'Loss': 1.61952}: 100%|██████████| 164/164 [00:25<00:00,  6.31it/s]


Test Accuracy: 0.83698


{'Epoch': 10, 'Loss': 1.61211}: 100%|██████████| 164/164 [00:26<00:00,  6.13it/s]


Test Accuracy: 0.84078


In [4]:

class Test:
	def __init__(self, train_x, train_y, test_x, test_y):
		self.train_x, self.train_y, self.test_x, self.test_y = train_x, train_y, test_x, test_y
		self.criterion = torch.nn.CrossEntropyLoss()
		
	def single_model(self, path):
		model = torch.load(path)
		model.eval()

		with torch.no_grad():

			train_loss, train_acc = 0, 0

			for batch_x, target in zip(self.train_x, self.train_y):
				output = model(batch_x)
				loss = self.criterion(output, target)
				output = torch.argmax(output, dim=1)
				acc = torch.sum(output == target)/target.shape[0]
				loss = loss.item()
				acc = acc.item()
				train_loss += loss
				train_acc += acc

			train_loss, train_acc = train_loss/self.train_x.shape[0], train_acc/self.train_x.shape[0]

			test_loss, test_acc = 0, 0

			for batch_x, target in zip(self.test_x, self.test_y):
				output = model(batch_x)
				loss = self.criterion(output, target)
				output = torch.argmax(output, dim=1)
				acc = torch.sum(output == target)/target.shape[0]
				loss = loss.item()
				acc = acc.item()
				test_loss += loss
				test_acc += acc

		test_loss, test_acc = test_loss/self.test_x.shape[0], test_acc/self.test_x.shape[0]

		return [train_loss, train_acc, test_loss, test_acc]

	def analyse_type(self, filename):
		model_names = os.listdir(filename)
		models = np.array([filename+i for i in model_names])
		model_names_index = np.argsort(np.array([int(i[len('model_epoch_'):-3]) for i in model_names]))
		models = models[model_names_index]
		performance = []
		for model in models:
			performance.append(self.single_model(model))
		performance = pd.DataFrame(np.array(performance))
		performance.columns = ['train_loss', 'train_acc', 'test_loss', 'test_acc']
		performance.to_csv('./results/performance_metrics/'+filename[len('./results/models/'):-1]+'.csv', index=False)

		performance = performance.values
		best_index = np.argmin(performance.T[2])
		return performance[best_index]

	def analyse_all(self):
		bar = tqdm(total=3*4*3)
		all_best_performances = []
		with bar:
			for local_epochs in [1, 3, 5]:
				for r in [0.5, 0.667, 0.833, 1.0]:
					for precision in [5, 6, 7]:
						filename = "./results/models/saved_models_local_epochs_"+str(local_epochs)+"_r_"+str(r).replace('.', '_')+"_precision_"+str(precision)+"/"
						best = self.analyse_type(filename)
						best = [local_epochs, r, precision] + [i for i in best]
						all_best_performances.append(best)
						bar.update(1)
		all_best_performances = pd.DataFrame(np.array(all_best_performances))
		all_best_performances.columns = ['local_epochs', 'r', 'precision', 'train_loss', 'train_acc', 'test_loss', 'test_acc']
		all_best_performances.to_csv('./results/best_performances.csv', index=False)

	def image_beautifier(self):

		image_names = sorted(['./results/'+i for i in os.listdir('./results/') if '.png' in i])
		for names in [image_names[i:i+4] for i in range(0, 12, 4)]:
			images = [Image.open(x) for x in names]
			widths, heights = zip(*(i.size for i in images))

			total_width = sum(widths)
			max_height = max(heights)

			new_im = Image.new('RGB', (total_width, max_height))

			x_offset = 0
			for im in images:
				new_im.paste(im, (x_offset,0))
				x_offset += im.size[0]

			name = names[0][len('./results/'):names[0].index('__')]
			new_im.save(name+'_variations.png')

		### Resizing for actual use

		for image in [i for i in os.listdir() if '_variations.png' in i]:
			img = cv2.resize(cv2.imread(image), (1280, 240))
			cv2.imwrite(image, img)

	def image(self, performances, names, pic_name):
		for i,name in enumerate(['train_loss', 'train_acc', 'test_loss', 'test_acc']):
			plt.cla()
			for j,performance in enumerate(performances):
				plt.plot(np.arange(10), performance.T[i], label=names[j])
			plt.legend()
			if i%2==1:
				plt.ylim([-0.01, 1.01])
			plt.title(name+' - '+pic_name)
			plt.savefig('./results/'+pic_name+'__'+name+'_analysis.png')

	def image_generator(self):
		performance = pd.read_csv('./results/best_performances.csv')
		features = performance.columns
		performance = performance.values
		best_index = np.argmin(performance.T[-2])
		print("Best Performance By: ", {i:j for i,j in zip(features, performance[best_index])})

		local_epochs, r, precision = performance[best_index][:3]
		local_epochs = int(local_epochs)
		precision = int(precision)

		### Local Epochs
		print("Analysing local_epochs with r and precision fixed to", r, precision, "respectively.")
		performances = []
		for local_epoch in [1, 3, 5]:
			filename = "./results/performance_metrics/saved_models_local_epochs_"+str(local_epoch)+"_r_"+str(r).replace('.', '_')+"_precision_"+str(precision)+".csv"
			performance = pd.read_csv(filename)
			performance = performance.values
			performances.append(performance)
		performances = np.array(performances)
		self.image(performances, names=['local_epochs='+str(i) for i in [1, 3, 5]], pic_name='local_epochs')

		### r
		print("Analysing local_epochs with local_epochs and precision fixed to", local_epochs, precision, "respectively.")
		performances = []
		for r_id in [0.5, 0.667, 0.833, 1.0]:
			filename = "./results/performance_metrics/saved_models_local_epochs_"+str(local_epochs)+"_r_"+str(r_id).replace('.', '_')+"_precision_"+str(precision)+".csv"
			performance = pd.read_csv(filename)
			performance = performance.values
			performances.append(performance)
		performances = np.array(performances)
		self.image(performances, names=['r='+str(i) for i in [0.5, 0.667, 0.833, 1.0]], pic_name='r')

		### r
		print("Analysing local_epochs with local_epochs and r fixed to", local_epochs, r, "respectively.")
		performances = []
		for p in [5, 6, 7]:
			filename = "./results/performance_metrics/saved_models_local_epochs_"+str(local_epochs)+"_r_"+str(r).replace('.', '_')+"_precision_"+str(p)+".csv"
			performance = pd.read_csv(filename)
			performance = performance.values
			performances.append(performance)
		performances = np.array(performances)
		self.image(performances, names=['precision='+str(i) for i in [5, 6, 7]], pic_name='precision')

		self.image_beautifier()



In [5]:
train_x, train_y, test_x, test_y = Dataset().load_csv()
test = Test(train_x, train_y, test_x, test_y)
test.analyse_all()
# test.image_generator()

  0%|          | 0/36 [00:00<?, ?it/s]


FileNotFoundError: [WinError 3] The system cannot find the path specified: './results/models/saved_models_local_epochs_1_r_0_5_precision_5/'

In [18]:
data = pd.read_csv(rf'E:\MS_Thesis\Hierarchical_quantization\FedPAQ-MNIST-implemenation-main\results\best_performances.csv')
data.sort_values(by='test_acc',ascending=False)

,local_epochs,r,precision,train_loss,train_acc,test_loss,test_acc
23,3.0,1.000,7.0,1.508578,0.954411,1.514189,0.948753
27,5.0,0.667,5.0,1.511121,0.951744,1.515928,0.946746
22,3.0,1.000,6.0,1.512752,0.951553,1.518188,0.946567
32,5.0,0.833,7.0,1.510649,0.952672,1.516274,0.946495
18,3.0,0.833,5.0,1.512172,0.951243,1.518068,0.946173
25,5.0,0.500,6.0,1.515957,0.947147,1.519424,0.944667
19,3.0,0.833,6.0,1.514992,0.949385,1.521292,0.944022
34,5.0,1.000,6.0,1.514191,0.948766,1.519685,0.943700
26,5.0,0.500,7.0,1.516056,0.947790,1.520447,0.943341
21,3.0,1.000,5.0,1.516362,0.947480,1.522686,0.941335
